# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [52]:
import pandas as pd
        
df = pd.read_csv('../../../../Datasets/previsao_de_renda.csv', sep = ',')

In [53]:
df.info()

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

#### 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

In [54]:
import statsmodels.api as sm
import numpy as np
import statsmodels.formula.api as smf
import patsy    

qualitative = df.select_dtypes(include='object').columns    
qualitative = qualitative.drop('data_ref')
print(qualitative)

In [55]:
# mantenha sempre a categoria mais frequente como casela de referência

#achando a categoria mais frequente
most_frequent = df[qualitative].mode().iloc[0]
print(most_frequent)
    

In [56]:

formula = 'np.log(renda) ~ ' + ' + '.join(
    [f"C({col}, Treatment(reference='{most_frequent[col]}'))" for col in qualitative]
)

# Criar os arrays de design
y, X = patsy.dmatrices(formula, df, return_type='dataframe')

In [57]:

model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

#### 2. Remova a variável menos significante e analise:

In [58]:
qualitative = qualitative.drop('estado_civil')

formula = 'np.log(renda) ~ ' + ' + '.join(
    [f"C({col}, Treatment(reference='{most_frequent[col]}'))" for col in qualitative]
)

# Criar os arrays de design
y, X = patsy.dmatrices(formula, df, return_type='dataframe')
model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

#### 3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

In [59]:
# agrupando dummies com menos de 5% de significância

# Identificar as variáveis dummies com p-valores maiores que 0.05
p_values = result.pvalues
high_p_values = p_values[p_values > 0.05]
high_p_dummies = high_p_values.index.tolist()

X_reduced = X.drop(columns=high_p_dummies)

# Ajustar o modelo novamente com as variáveis restantes
model_reduced = sm.OLS(y, X_reduced)
result_reduced = model_reduced.fit()
result_reduced.summary()

# Conclusão

    Sim o modelo melhorou, o F-statistic aumentou e o AIC e BIC diminuiram enquanto o R-squared ajustado se manteve o mesmo. Embora o modelo em si é bem ruim.